In [2]:
import os
import numpy as np
from tqdm import tqdm_notebook as tqdm
from phathom import io
from phathom.io import conversion
from phathom import utils

In [3]:
working_dir = '/media/jswaney/Drive/Justin/organoid_etango'

# 3D tiff to zarr

In [ ]:
paths = ['syto16.tif', 'sox2.tif', 'tbr1.tif']
chunks = (64, 64, 64)

for path in tqdm(paths):
    img = io.tiff.imread(os.path.join(working_dir, path))
    baio.zarr.new_zarrh.split()[0]
    z_arr = io.zarr.new_zarr(os.path.join(working_dir, basename+'.zarr'), 
                             img.shape,
                             chunks=chunks,
                             dtype=img.dtype))
    z_arr[:] = img
    del img

# 2D tiffs to zarr

In [13]:
folders = [#'syto16_clahe', 
           #'sox2_clahe', 
           'tbr1']
suffix = '.zarr'
chunks = (64, 64, 64)

for folder in tqdm(folders):
    tif_dir = os.path.join(working_dir, folder)
    zarr_path = os.path.join(working_dir, folder + suffix)
    conversion.tifs_to_zarr(tif_dir, 
                        zarr_path, 
                        chunks, 
                        in_memory=False, 
                        nb_workers=2)

Move these chunked arrays to subfolder called `1_1_1` as first mipmap level

In [4]:
import zarr
from precomputed_tif import ZarrStack

In [ ]:
src_paths = ['syto16_clahe.zarr/1_1_1', 
             'sox2_clahe.zarr/1_1_1', 
             'tbr1_clahe.zarr/1_1_1']
dest_paths = ['syto16_clahe.zarr', 
             'sox2_clahe.zarr', 
             'tbr1_clahe.zarr']
n_levels = 4

for src_path, dest_path in tqdm(zip(src_paths, dest_paths)):
    src = zarr.NestedDirectoryStore(os.path.join(working_dir, src_path))
    dest = os.path.join(working_dir, dest_path)
    stack = ZarrStack(src, dest)
    stack.write_info_file(n_levels)
    for level in range(2, n_levels+1):
        stack.write_level_n(level)

In [6]:
n_levels = 4

syto16_stack.write_info_file(n_levels)
sox2_stack.write_info_file(n_levels)

for level in range(2, n_levels+1):
    syto16_stack.write_level_n(level)
    sox2_stack.write_level_n(level)

100%|██████████| 162/162 [00:09<00:00, 17.08it/s]


# downsample 2D tiffs

Used for graph-cuts based ventricle detection

In [ ]:
input_dirs = ['syto16_clahe', 'sox2_clahe', 'tbr1_clahe']
suffix = ['_4x']
factors = (4, 4)

for input_dir in tqdm(input_dirs):
    output_dir = input_dir + suffix
    conversion.downscale_tif_batch(input_dir, factors, output_dir)

# Extract small image to test

In [7]:
z_arr_paths = ['syto16_clahe.zarr/1_1_1', 
               'sox2.zarr/1_1_1', 
               'tbr1.zarr']
start = np.array((256, 2048, 2048))
shape = np.array((256, 512, 512))

boxes = []
stop = start + shape
for z_arr_path in z_arr_paths:
    z_arr = io.zarr.open(os.path.join(working_dir, z_arr_path))
    box = utils.extract_box(z_arr, start, stop)
    boxes.append(box)

In [8]:
new_paths = ['syto16_test2.zarr',
             'sox2_test2.zarr',
             'trb1_test2.zarr']
chunks = (64, 64, 64)

for new_path, box in zip(new_paths, boxes):
    path = os.path.join(working_dir, new_path)
    new_z_arr = io.zarr.new_zarr(path, box.shape, chunks, box.dtype)
    new_z_arr[:] = box